In [76]:
## Public Records Request Notes
## First PRR received on 3/29/2022 for data that spans 2018 - 3/29/2022
## Second PPR received on 11/22/2022 for data that spans 3/29/2022 - 11/22/2022

In [77]:
import pandas as pd
import re

In [78]:
from lib import standardize_item_numbers

def clean():
    dfa = pd.read_csv("../data/police_reports/electronic_police_report_2018_2022.csv")
    dfa = dfa.pipe(standardize_item_numbers, ["item_number"])
    
    dfb = pd.read_csv("../data/real_time_crime_center/rtcc.csv")
    dfb = dfb.pipe(standardize_item_numbers, ["item_number"])
    dfb["rtcc_footage_requested"] = "yes"

    df = pd.merge(dfb, dfa, on="item_number")
    return df

In [79]:
def drop_rows_missing_offender_race(df):
    df.loc[:, "offender_race"] = df.offender_race.fillna("").str.lower().str.strip()\
    .str.replace("unknown", "", regex=False)\
    # .str.replace("hispanic", "", regex=False)\
    # .str.replace("asian", "", regex=False)\
    # .str.replace(r"amer\. ind\.", "", regex=True)
    return df[~((df.offender_race == ""))]

def drop_rows_missing_victim_race(df):
    df.loc[:, "victim_race"] = df.victim_race.fillna("").str.lower().str.strip()\
    .str.replace("unknown", "", regex=False)\
    .str.replace(r"native hawaiian or other pacific islander", "", regex=False)
    # .str.replace("hispanic", "", regex=False)\
    # .str.replace("asian", "", regex=False)\
    # .str.replace(r"amer\. ind\.", "", regex=True)
    return df[~((df.victim_race == ""))]


def filter_offender_gender(df):
    df.loc[:, "offender_gender"] = (
        df.offender_gender.str.lower().str.strip().fillna("").str.replace("unknown", "", regex=False)
    )
    return df[~((df.offender_gender == ""))]



def filter_victim_gender(df):
    df.loc[:, "victim_gender"] = (
        df.offender_gender.str.lower().str.strip().fillna("").str.replace("unknown", "", regex=False)
    )
    return df[~((df.victim_gender == ""))]


def filter_arrested(df):
    df.loc[:, "offenderstatus"] = df.offenderstatus.str.lower().str.strip().fillna("")
    df = df[df.offenderstatus.isin(["arrested"])]
    return df[~((df.offenderstatus == ""))]


def filter_year(df):
    # df = df[df.year.astype(str).isin(["2018", r"2019", r"2020", r"2021\.", r"2022\."])]
    return df[~((df.year.fillna("") == ""))]
  
  
def drop_rows_missing_charge_desc(df):
    df.loc[:, "charge_description"] = df.charge_description.str.lower().str.strip().fillna("")
    return df[~((df.charge_description == ""))]

In [80]:
df = clean()
df.shape
df.to_csv("../data/real_time_crime_center/rtcc_joined_with_police_reports_2018-2022.csv", index=False)

C:\Users\PC\AppData\Local\Temp\ipykernel_1324\2761447203.py:4: DtypeWarning: Columns (13,23,24) have mixed types. Specify dtype option on import or set low_memory=False.
  dfa = pd.read_csv("../data/police_reports/electronic_police_report_2018_2022.csv")


In [81]:
df = df.pipe(filter_year).pipe(drop_rows_missing_offender_race).pipe(filter_offender_gender)\
    .drop_duplicates(subset=["offender_race", "offender_gender", "offender_age", "item_number"])

In [82]:
df.columns

Index(['item_number', 'Signal Code', 'rtcc_footage_requested', 'district',
       'location', 'disposition', 'signal_type', 'signal_description',
       'occurred_date_time', 'charge_code', 'charge_description',
       'offender_race', 'offender_gender', 'offender_age', 'offender_number',
       'person_type', 'victim_race', 'victim_gender', 'victim_age',
       'victim_number', 'victim_fatal_status', 'hate_crime', 'report_type',
       'year', 'offenderid', 'offenderstatus', 'persontype'],
      dtype='object')

In [83]:
df.loc[:, "rtcc_footage_requested"] = df.rtcc_footage_requested.fillna("no")

In [84]:
df["rtcc_footage_request_tally"] = 1

In [85]:
df.rtcc_footage_requested.value_counts()

yes    4815
Name: rtcc_footage_requested, dtype: int64

In [86]:
rtcc = df[df.rtcc_footage_requested.isin(["yes"])]

In [87]:
rtcc_district = rtcc[["district", "rtcc_footage_request_tally",]]
rtcc_district_perc = rtcc_district.groupby(["district",]).rtcc_footage_request_tally.count()/len(rtcc_district)
rtcc_district_counts = rtcc_district.groupby(["district",]).rtcc_footage_request_tally.count()
rtcc_district_counts = pd.DataFrame(rtcc_district_counts).reset_index()
rtcc_district_perc = pd.DataFrame(rtcc_district_perc).reset_index()
rtcc_district = pd.concat([rtcc_district_perc, rtcc_district_counts])
# rtcc_district
# rtcc_district.to_excel("../data/excel/rtcc_requests_district.xlsx")

In [88]:
rtcc = rtcc[["offender_race", "offender_gender", "rtcc_footage_request_tally",]]
rtcc = rtcc.groupby(["offender_race", "offender_gender"]).rtcc_footage_request_tally.count()
rtcc = pd.DataFrame(rtcc).reset_index()

In [89]:
# rtcc.loc[(rtcc.offender_race == "black"), "population"] = "105498"
# rtcc.loc[(rtcc.offender_race == "white"), "population"] = "65756"
# rtcc.loc[(rtcc.offender_race == "hispanic"), "population"] = "11205"
# rtcc.loc[(rtcc.offender_race == "amer. ind."), "population"] = "360"
# rtcc.loc[(rtcc.offender_race == "asian"), "population"] = "5499"


# rtcc.loc[(rtcc.victim_race == "black"), "population"] = "105498"
# rtcc.loc[(rtcc.victim_race == "white"), "population"] = "65756"
# rtcc.loc[(rtcc.victim_race == "hispanic"), "population"] = "11205"
# rtcc.loc[(rtcc.victim_race == "amer. ind."), "population"] = "360"
# rtcc.loc[(rtcc.victim_race == "asian"), "population"] = "5499"

rtcc.loc[(rtcc.offender_race == "black") & (rtcc.offender_gender == "male"), "population"] = "52749"
rtcc.loc[(rtcc.offender_race == "black") & (rtcc.offender_gender == "female"), "population"] = "52749"
rtcc.loc[(rtcc.offender_race == "white") & (rtcc.offender_gender == "male"), "population"] = "32878"
rtcc.loc[(rtcc.offender_race == "white") & (rtcc.offender_gender == "female"), "population"] = "32878"
rtcc.loc[(rtcc.offender_race == "hispanic") & (rtcc.offender_gender == "male"), "population"] = "5602"
rtcc.loc[(rtcc.offender_race == "hispanic") & (rtcc.offender_gender == "female"), "population"] = "5602"
rtcc.loc[(rtcc.offender_race == "native american") & (rtcc.offender_gender == "male"), "population"] = "180"
rtcc.loc[(rtcc.offender_race == "native american") & (rtcc.offender_gender == "female"), "population"] = "180"
rtcc.loc[(rtcc.offender_race == "amer. ind.") & (rtcc.offender_gender == "male"), "population"] = "180"
rtcc.loc[(rtcc.offender_race == "amer. ind.") & (rtcc.offender_gender == "female"), "population"] = "180"
rtcc.loc[(rtcc.offender_race == "asian") & (rtcc.offender_gender == "male"), "population"] = "2750"
rtcc.loc[(rtcc.offender_race == "asian") & (rtcc.offender_gender == "female"), "population"] = "2750"

In [90]:
rtcc

,offender_race,offender_gender,rtcc_footage_request_tally,population
0,amer. ind.,female,1,180
1,amer. ind.,male,2,180
2,asian,male,8,2750
3,black,female,604,52749
4,black,male,3692,52749
5,hispanic,female,10,5602
6,hispanic,male,47,5602
7,native hawaiian or other pacific islander,female,1,NaN
8,white,female,92,32878
9,white,male,358,32878


In [91]:
def calc_rate(dfb_total, population=""):
    population = int(population)
    rate = (( dfb_total / population) * 1000)
    return rate

In [92]:
rtcc["rate_per_1000_persons"] = rtcc.apply(lambda x: calc_rate(x.rtcc_footage_request_tally, x.population), axis=1)
rtcc["year"] = "2018-2022"
rtcc

ValueError: cannot convert float NaN to integer

In [ ]:
# rtcc.to_excel("../data/excel/rtcc_requests_offender_by_race_and_gender.xlsx")

In [ ]:
rtcc_charges = df[df.rtcc_footage_requested.isin(["yes"])]

In [ ]:
rtcc_charges.columns

Index(['item_number', 'Signal Code', 'rtcc_footage_requested', 'Unnamed: 0',
       'district', 'location', 'disposition', 'signal_type',
       'signal_description', 'occurred_date_time', 'charge_code',
       'charge_description', 'offender_race', 'offender_gender',
       'offender_age', 'offender_number', 'person_type', 'victim_race',
       'victim_gender', 'victim_age', 'victim_number', 'victim_fatal_status',
       'hate_crime', 'report_type', 'year', 'offenderid', 'offenderstatus',
       'persontype', 'rtcc_footage_request_tally'],
      dtype='object')

In [ ]:
rtcc_charges.charge_description.unique()

array(['SECOND DEGREE MURDER', 'SIMPLE CRIMINAL DAMAGE TO PROPERTY',
       'ARMED ROBBERY', 'THEFT OF A MOTOR VEHICLE', 'BATTERY',
       'POSSESSION OF FIREARM BY JUVENILE', 'AGG. BATTERY',
       'AGGRAVATED ASSAULT WITH A FIREARM',
       'ILLEGAL POSSESSION OF STOLEN THINGS', 'SIMPLE BURGLARY',
       'SEXUAL BATTERY', 'SIMPLE ROBBERY', 'SIMPLE BATTERY',
       'SIMPLE BURGLARY (INHABITED DWELLING)', 'PURSE SNATCHING', nan,
       'ILLEGAL CARRYING OF WEAPON', 'OBSCENITY',
       'AGG. SECOND DEGREE BATTERY', 'CRIMINAL DAMAGE TO PROPERTY',
       'AGG. ASSAULT', 'ATTEMPT - PURSE SNATCHING',
       'ATTEMPT - SECOND DEGREEMURDER', 'PRINCIPAL TO ARMED ROBBERY',
       'ATTEMPT - ARMED ROBBERY', 'BATTERY OF A DATING PARTNER (SIMPLE)',
       'CARJACKING', 'USE OF FIREARM IN ROBBERY', 'AGG. CRIMINAL DAMAGE',
       'AGG. BURGLARY', 'THEFT', 'RECKLESS OPERATION OF A VEHICLE',
       'ILLEGAL DUMPING', 'DISTURBING THE PEACE',
       'POSSESSION OF FIREARM BY FELON', 'ILLEGAL USE OF WEAP

In [ ]:
# rtcc_charges.charge_description.value_counts().head(25).to_excel("../data/excel/rtcc_charges_charge_description.xlsx")

In [ ]:
# rtcc_charges.groupby("year").disposition.value_counts().to_excel("../data/excel/rtcc_charges_disposition.xlsx")

In [ ]:
rtcc_charges.loc[:, "offender_gender"] = rtcc_charges.offender_gender.str.lower().str.strip().fillna("")\
    .str.replace(r"unknown", "", regex=True)
rtcc_charges.offender_gender.unique()

array(['male', 'female'], dtype=object)

In [ ]:
charge = df[df.charge_description.isin(["ILLEGAL POSSESSION OF STOLEN THINGS"])]


blacks = charge[charge.offender_race.isin(["black"])]
blacks_perc = blacks.groupby(["offender_race", "charge_description"]).rtcc_footage_requested.value_counts()/len(blacks)
blacks_perc = pd.DataFrame(blacks_perc)

blacks_counts = blacks.groupby(["offender_race", "charge_description"]).rtcc_footage_requested.value_counts()
blacks_counts = pd.DataFrame(blacks_counts)


whites = charge[charge.offender_race.isin(["white"])]
whites_perc = whites.groupby(["offender_race", "charge_description"]).rtcc_footage_requested.value_counts()/len(whites)
whites_perc = pd.DataFrame(whites_perc)

whites_counts = whites.groupby(["offender_race", "charge_description"]).rtcc_footage_requested.value_counts()
whites_counts = pd.DataFrame(whites_counts)


df_concat = pd.DataFrame(pd.concat([blacks_perc, whites_perc, blacks_counts, whites_counts]))
df_concat.columns
# df_concat.sort_values("offender_race").to_excel("../data/excel/armed_robbery_rtcc_footage_request_by_illegal_posses.xlsx")
# df_concat
# df_concat.to_excel("../data/excel/armed_robbery_rtcc_footage_request_by_race_armed_robbery.xlsx")


Index(['rtcc_footage_requested'], dtype='object')